In [3]:
import tensorflow as tf

In [4]:
#张量 tensor 就是 多维数组 多维列表 阶：tensor的维度  0维 就是一个数字  

a = tf.constant([1,2])
b = tf.constant([3,4])

result = a + b
print(result)

'''
这就是一个计算图  只描述过程  不运算
'''

Tensor("add:0", shape=(2,), dtype=int32)


In [5]:
# 计算图：描述一个简单的神经传递

x = tf.constant([[1,2]])
w = tf.constant([[3],[4]])

y = tf.matmul(x,w)
print(y)

Tensor("MatMul:0", shape=(1, 1), dtype=int32)


In [6]:
# 绘画session：执行计算图中的节点运算
x = tf.constant([[1,2]])
w = tf.constant([[3],[4]])

y = tf.matmul(x,w)

with tf.Session() as sess:
    print(sess.run(y))

[[11]]


In [9]:
#初始化参数 ————随机生成

w = tf.Variable(tf.random_normal([2,3],stddev=2,mean=0,seed=1))  
#生成一个随机权重 random_normal 是代表正态分布 [2,3]是生成的形状 stddv是标准差  mean是均值  seed随机种子

#  tf.truncated_normal() 去除偏离值的正态分布
#  tf.random_uniform()  平均分布

#还有——tf.zeros全零  tf.ones全1  tf.fill定值数组  tf.constant直接给指

In [18]:
#  前向传播
'''
神经网络：
    准备数据集，提取特征，喂给网络
    搭建NN，输入到输出，先搭建计算图然后执行session（前向传播）
    迭代优化参数（反向传播）
    使用模型预测
    
'''

#定义输入和参数（直接输入 or 占位符）
#x = tf.constant([[0.7,0.5]])
#x = tf.placeholder(tf.float32,shape=(1,2))
#可以用占位符,用了占位符就要在最后run的时候喂数据

#假如我们不知道要喂多少组数据
x = tf.placeholder(tf.float32,shape=(None,2))
#下面feed_dict 就可以随意增加样本数量了


w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

#定义前向传播过程
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

#用session计算
with  tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('w1',sess.run(w1),"w2",sess.run(w2))
    #print(sess.run(y))
    #print(sess.run(y,feed_dict={x:[[0.7,0.5]]}))
    print(sess.run(y,feed_dict={x:[[0.7,0.5],[0.2,0.3],[0.3,0.4],[0.4,0.5]]}))

w1 [[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]] w2 [[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
[[3.0904665]]


In [30]:
# 反向传播
'''
利用梯度下降，减少损失函数

loss：预测值跟实际答案的差距

均方误差MSE

loss = tf.reduce_mean(tf.square(y,y^))

反向传播：以最小化loss为优化目标(三个下降算法，)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
train_step = tf.train.MomentumOptimizer(learning_rate).minimize(loss)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

神经网络：
    准备数据集，提取特征，喂给网络
    搭建NN，输入到输出，先搭建计算图然后执行session（前向传播）
    迭代优化参数（反向传播）
    使用模型预测
'''
import numpy as np

batch_size = 8
seed = 23455

#基于seed生成随机数
rng =  np.random.RandomState(seed)

# 随机数返回32 行 2列 的矩阵  表示32组 体积和重量  作为输入数据集
X = rng.rand(32,2)

#如果小于1给Y赋值 1(合格)  如果不小于1给Y赋值0（不合格）   （逻辑判断）
#结果作为数据集的标签(答案)
Y = [[int(x0+x1<1)] for (x0,x1) in X]
print(X,Y)

#       1-定义神经网络输入、参数、输出，定义前向传播过程
x = tf.placeholder(tf.float32,shape=(None,2))
y_ = tf.placeholder(tf.float32,shape=(None,1))

w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

#       2-定义loss函数以及反向传播
loss = tf.reduce_mean(tf.square(y-y_))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
#train_step = tf.train.MomentumOptimizer(learning_rate).minimize(loss)
#train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)


#       3-生成session，训练steps轮
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    #输出目前未被训练的参数 w1 w2
    print('w1',sess.run(w1),'w2',sess.run(w2))
    print('\n')
    
    #训练模型
    steps = 3000
    for i in range(steps):
        start = (i*batch_size)%32  #这里就是minibatch的用法
        end = start + batch_size
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        if i %500 ==0:
            total_loss = sess.run(loss,feed_dict={x:X,y_:Y})
            print('当前loss值',total_loss)
    
    #训练结束后再次输出参数
    print('w1',sess.run(w1),'w2',sess.run(w2))


    

[[0.83494319 0.11482951]
 [0.66899751 0.46594987]
 [0.60181666 0.58838408]
 [0.31836656 0.20502072]
 [0.87043944 0.02679395]
 [0.41539811 0.43938369]
 [0.68635684 0.24833404]
 [0.97315228 0.68541849]
 [0.03081617 0.89479913]
 [0.24665715 0.28584862]
 [0.31375667 0.47718349]
 [0.56689254 0.77079148]
 [0.7321604  0.35828963]
 [0.15724842 0.94294584]
 [0.34933722 0.84634483]
 [0.50304053 0.81299619]
 [0.23869886 0.9895604 ]
 [0.4636501  0.32531094]
 [0.36510487 0.97365522]
 [0.73350238 0.83833013]
 [0.61810158 0.12580353]
 [0.59274817 0.18779828]
 [0.87150299 0.34679501]
 [0.25883219 0.50002932]
 [0.75690948 0.83429824]
 [0.29316649 0.05646578]
 [0.10409134 0.88235166]
 [0.06727785 0.57784761]
 [0.38492705 0.48384792]
 [0.69234428 0.19687348]
 [0.42783492 0.73416985]
 [0.09696069 0.04883936]] [[1], [0], [0], [1], [1], [1], [1], [0], [1], [1], [1], [0], [0], [0], [0], [0], [0], [1], [0], [0], [1], [1], [0], [1], [0], [1], [1], [1], [1], [1], [0], [1]]
w1 [[-0.8113182   1.4845988   0.065329

In [ ]:
"""
准备：
    import 模块
    常量定义
    生成数据集
    
前向传播：
    定义输入 x=  y_=
    定义参数 w1=  w2=
    定义输出  a= y=
    
反向传播：定义损失函数、反向传播优化方法
    loss = 
    train_step = SGD MGD Adam
    
生成session 迭代：
    with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    steps = 
    for i in range(steps):
        strat =
        end = 
        sess.run(train_step,feed_dict=:{})
    

"""